# OBGN- Mag DataSet Training

In [1]:
from ogb.nodeproppred import DglNodePropPredDataset, Evaluator
from dgl import AddReverse, Compose, ToSimple
import dgl

import torch as th
import torch.nn.functional as F

from tqdm import tqdm
import itertools

/datasets/vjawa/miniconda3/envs/cugraph_dgl_dev_oct_5/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
### Local Imports from this folder
from logger import Logger

## Step 1.a:  Load the Graph

In [3]:
def load_dgl_graph(paper_dim_to_load=10):
    dataset = DglNodePropPredDataset(name="ogbn-mag", root='/datasets/vjawa/gnn/')
    split_idx = dataset.get_idx_split()
    # graph: dgl graph object, label: torch tensor of shape (num_nodes, num_tasks)
    g, labels = dataset[0]        
    # Update to paper_dim_to_load to save on GPU memory for non distributed versions
    # We need to clear g.ndata to do this
    ndata = {k: v for k, v in g.ndata.items()}
    g.ndata.clear()        
    ndata['feat']['paper'] = ndata['feat']['paper'][:,:paper_dim_to_load]
    g.ndata.update(ndata)
    
    labels = labels["paper"].flatten()
    transform = Compose([ToSimple(), AddReverse()])
    g = transform(g)
    print("Loaded graph: {}".format(g))
    return g, labels, dataset.num_classes, split_idx


In [4]:
load_d_feature = 10
device = 'cuda'

g, labels, num_classes, split_idx = load_dgl_graph(load_d_feature)
assert g.ndata['feat']['paper'].shape[1] == load_d_feature


g = g.to(device)

Loaded graph: Graph(num_nodes={'author': 1134649, 'field_of_study': 59965, 'institution': 8740, 'paper': 736389},
      num_edges={('author', 'affiliated_with', 'institution'): 1043998, ('author', 'writes', 'paper'): 7145660, ('field_of_study', 'rev_has_topic', 'paper'): 7505078, ('institution', 'rev_affiliated_with', 'author'): 1043998, ('paper', 'cites', 'paper'): 10832542, ('paper', 'has_topic', 'field_of_study'): 7505078, ('paper', 'rev_writes', 'author'): 7145660},
      metagraph=[('author', 'institution', 'affiliated_with'), ('author', 'paper', 'writes'), ('institution', 'author', 'rev_affiliated_with'), ('paper', 'paper', 'cites'), ('paper', 'field_of_study', 'has_topic'), ('paper', 'author', 'rev_writes'), ('field_of_study', 'paper', 'rev_has_topic')])


## Step 1.b Convert Graph from DGL Graph to cugraph

In [5]:
### Local Imports from this folder
from dgl_cugraph_conversion_utils import add_edges, add_nodes, add_ndata

In [ ]:
def graphstore_from_hetrograph(gs, g):
    num_nodes_dict = {ntype: g.num_nodes(ntype) for ntype in g.ntypes}
    add_nodes(gs, g, num_nodes_dict)
    print("Added Nodes")
    add_edges(gs, g, num_nodes_dict)
    print("Added Edges")
    add_ndata(gs, g, num_nodes_dict)
    print("Added ndata")
    #TODO: 
    # add_edata

gs = dgl.contrib.cugraph.CuGraphStorage(idtype=g.idtype)
graphstore_from_hetrograph(gs, g)

Added Nodes
Added Edges


#### Verify Similar Structure

In [ ]:
## Testing Similar Structure
from dgl_cugraph_conversion_utils import assert_same_num_nodes, assert_same_num_edges
gs.num_edges(etype='affiliated_with')==g.num_edges(etype='affiliated_with')
assert_same_num_edges(gs, g)
assert_same_num_nodes(gs, g)

## Step 1.c Initiate Sampler and Train_Loader

In [ ]:
#### Run only on 10_000 samples to test the pipeline
n_samples = 5_000
subset_split_idx = {'train': {k: v[:n_samples].to(device) for k,v in split_idx['train'].items()},
                   'valid' : {k: v[:n_samples].to(device) for k,v in split_idx['valid'].items()},
                    'test' : {k: v[:n_samples].to(device) for k,v in split_idx['test'].items()},
                   }


sampler = dgl.dataloading.MultiLayerNeighborSampler([25, 20], prefetch_node_feats={'paper':['feat']})
train_loader = dgl.dataloading.DataLoader(
    gs,
    #split_idx,
    subset_split_idx["train"],
    sampler,
    batch_size=1024,
    shuffle=True,
    num_workers=0,
    device='cuda'
)

## Step 2. Initate a Model

###  Traing HyperParameters

In [ ]:
from model import  rel_graph_embed,extract_embed
from model import EntityClassify

In [ ]:
nruns = 2
logger = Logger(nruns)
device = 'cuda'

In [ ]:
embedding_shape = g.ndata['feat']['paper'].shape[1]
embed_layer = rel_graph_embed(g, embedding_shape).to(device)
model = EntityClassify(g, embedding_shape, num_classes).to(device)

print(
    f"Number of embedding parameters: {sum(p.numel() for p in embed_layer.parameters())}"
)
print(
    f"Number of model parameters: {sum(p.numel() for p in model.parameters())}"
)

In [ ]:
g

## Model Training

In [64]:
def train(
    g,
    model,
    node_embed,
    optimizer,
    train_loader,
    split_idx,
    labels,
    logger,
    device,
    run,
):
    print("start training...")
    category = "paper"
    for epoch in range(2):
        num_train = split_idx["train"][category].shape[0]
        pbar = tqdm(total=num_train)
        pbar.set_description(f"Epoch {epoch:02d}")
        model.train()

        total_loss = 0

        for input_nodes, seeds, blocks in train_loader:
            blocks = [blk.to(device) for blk in blocks]
            seeds = seeds[
                category
            ]  # we only predict the nodes with type "category"
            batch_size = seeds.shape[0]

            emb = extract_embed(node_embed, input_nodes)
            
            feat = blocks[0].srcdata['feat']['paper']
            #label = subgs[-1].dstdata['label']
    
            # Add the batch's raw "paper" features
            emb.update(
                {"paper": feat}
                #{"paper": g.ndata["feat"]["paper"][input_nodes["paper"]]}
            )

            emb = {k: e.to(device) for k, e in emb.items()}
            lbl = labels[seeds].to(device)

            optimizer.zero_grad()
            logits = model(emb, blocks)[category]

            y_hat = logits.log_softmax(dim=-1)
            loss = F.nll_loss(y_hat, lbl)
            loss.backward()
            optimizer.step()

            total_loss += loss.item() * batch_size
            pbar.update(batch_size)
            
        pbar.close()
        loss = total_loss / num_train

        result = test(g, model, node_embed, labels, device, split_idx)
        logger.add_result(run, result)
        train_acc, valid_acc, test_acc = result
        print(
            f"Run: {run + 1:02d}, "
            f"Epoch: {epoch +1 :02d}, "
            f"Loss: {loss:.4f}, "
            f"Train: {100 * train_acc:.2f}%, "
            f"Valid: {100 * valid_acc:.2f}%, "
            f"Test: {100 * test_acc:.2f}%"
        )

    return logger


@th.no_grad()
def test(g, model, node_embed, y_true, device, split_idx):
    model.eval()
    category = "paper"
    evaluator = Evaluator(name="ogbn-mag")

    #dev type
    d_type = g.device.type
    
    # 2 GNN layers
    sampler = dgl.dataloading.MultiLayerFullNeighborSampler(2,  prefetch_node_feats={'paper':['feat']})
    loader = dgl.dataloading.DataLoader(
        g,
        {"paper": th.arange(g.num_nodes("paper")).to(d_type)},
        sampler,
        batch_size=16384,
        shuffle=False,
        num_workers=0,
    )

    pbar = tqdm(total=y_true.size(0))
    pbar.set_description(f"Inference")

    y_hats = list()
    
    for input_nodes, seeds, blocks in loader:
        blocks = [blk.to(device) for blk in blocks]
        seeds = seeds[
            category
        ]  # we only predict the nodes with type "category"
        batch_size = seeds.shape[0]

        emb = extract_embed(node_embed, input_nodes)
        # Get the batch's raw "paper" features
        
        ## VJawa: Use prefetched feat
        #emb.update({"paper": g.ndata["feat"]["paper"][input_nodes["paper"]]})
        ## prefetch_feat
        feat = blocks[0].srcdata['feat']['paper']
        emb.update({"paper": feat})


            
        emb = {k: e.to(device) for k, e in emb.items()}

        logits = model(emb, blocks)[category]
        y_hat = logits.log_softmax(dim=-1).argmax(dim=1, keepdims=True)
        y_hats.append(y_hat.cpu())

        pbar.update(batch_size)

        

    pbar.close()

    y_pred = th.cat(y_hats, dim=0)
    y_pred_rows = y_pred.shape[0]
    
    y_true = th.unsqueeze(y_true[:y_pred_rows], 1)
    
    
    train_split_idx = split_idx["train"]["paper"]
    valid_split_idx = split_idx["valid"]["paper"]
    test_split_idx =  split_idx["test"]["paper"]
    
    ### I only want to calculate over the rows i had
    

    train_acc = evaluator.eval(
        {
            "y_true": y_true[train_split_idx],
            "y_pred": y_pred[train_split_idx],
        }
    )["acc"]
    valid_acc = evaluator.eval(
        {
            "y_true": y_true[valid_split_idx],
            "y_pred": y_pred[valid_split_idx],
        }
    )["acc"]
    test_acc = evaluator.eval(
        {
            "y_true": y_true[test_split_idx],
            "y_pred": y_pred[test_split_idx],
        }
    )["acc"]

    return train_acc, valid_acc, test_acc

In [65]:
frontier_gs = gs.sample_neighbors(subset_split_idx['train'], fanout=25)
eid_gs = frontier_gs.edata[dgl.EID]

In [66]:
seed_nodes_gs = subset_split_idx['train']
for fanout in [20, 1]:
    frontier_g = gs.sample_neighbors(seed_nodes_gs, fanout=fanout)
    eid_g = frontier_g.edata[dgl.EID]
    eid = frontier.edata[dgl.EID]
    block = dgl.to_block(frontier, seed_nodes)
    block.edata[dgl.EID] = eid
    seed_nodes_gs = block.srcdata[dgl.NID]
    break

In [67]:
seed_nodes_gs

{'author': tensor([ 45575,  57763,  59351,  ...,  28288, 249159, 403467], device='cuda:0'),
 'field_of_study': tensor([  246,  2399,  2809,  ..., 15214,  5758, 25716], device='cuda:0'),
 'institution': tensor([], device='cuda:0', dtype=torch.int64),
 'paper': tensor([     0,      1,      2,  ..., 110398, 133077, 713876], device='cuda:0')}

In [ ]:
seed_nodes = subset_split_idx['train']
for fanout in [20, 1]:
    frontier_g = g.sample_neighbors(seed_nodes, fanout=fanout)
    eid_g = frontier_g.edata[dgl.EID]
    eid = frontier.edata[dgl.EID]
    block = dgl.to_block(frontier, seed_nodes)
    block.edata[dgl.EID] = eid
    seed_nodes = block.srcdata[dgl.NID]
    break

/home/nfs/vjawa/dgl/dgl/src/graph/transform/cuda/../../../runtime/cuda/cuda_hashtable.cuh:167: IdType dgl::runtime::cuda::DeviceOrderedHashTable<IdType>::SearchForPosition(IdType) const [with IdType = signed long]: block: [37,1,0], thread: [96,0,0] Assertion `table_[pos].key != kEmptyKey` failed.
/home/nfs/vjawa/dgl/dgl/src/graph/transform/cuda/../../../runtime/cuda/cuda_hashtable.cuh:167: IdType dgl::runtime::cuda::DeviceOrderedHashTable<IdType>::SearchForPosition(IdType) const [with IdType = signed long]: block: [37,1,0], thread: [97,0,0] Assertion `table_[pos].key != kEmptyKey` failed.
/home/nfs/vjawa/dgl/dgl/src/graph/transform/cuda/../../../runtime/cuda/cuda_hashtable.cuh:167: IdType dgl::runtime::cuda::DeviceOrderedHashTable<IdType>::SearchForPosition(IdType) const [with IdType = signed long]: block: [37,1,0], thread: [98,0,0] Assertion `table_[pos].key != kEmptyKey

In [ ]:
seed_nodes_gs

In [58]:
seed_nodes['paper'].shape

torch.Size([326151])

In [56]:
for run in range(nruns):
    embed_layer.reset_parameters()
    model.reset_parameters()

    # optimizer
    all_params = itertools.chain(
        model.parameters(), embed_layer.parameters()
    )
    optimizer = th.optim.Adam(all_params, lr=0.01)

    logger = train(
        gs,
        model,
        embed_layer,
        optimizer,
        train_loader,
        subset_split_idx,
        #TODO: Change to split_idx,
        labels,
        logger,
        device,
        run,
    )
    logger.print_statistics(run)

print("Final performance: ")
logger.print_statistics()

start training...





  0%|          | 0/50000 [00:00<?, ?it/s]


Epoch 00:   0%|          | 0/50000 [00:00<?, ?it/s]

RuntimeError: non-success value returned from cugraph_uniform_neighbor_sample: CUGRAPH_UNKNOWN_ERROR cuGraph failure at file=/home/nfs/vjawa/dgl/cugraph/cpp/src/prims/per_v_random_select_transform_outgoing_e.cuh line=304: Invalid input argument: frontier includes out-of-range keys.
Obtained 64 stack frames
#0 in /datasets/vjawa/miniconda3/envs/cugraph_dgl_dev_oct_5/lib/libcugraph.so(+0x8e1a2b) [0x7f803ff41a2b]
#1 in /datasets/vjawa/miniconda3/envs/cugraph_dgl_dev_oct_5/lib/libcugraph.so(+0xb21c6d) [0x7f8040181c6d]
#2 in /datasets/vjawa/miniconda3/envs/cugraph_dgl_dev_oct_5/lib/libcugraph.so(+0x170f4c4) [0x7f8040d6f4c4]
#3 in /datasets/vjawa/miniconda3/envs/cugraph_dgl_dev_oct_5/lib/libcugraph.so(_ZN7cugraph6detail12sample_edgesINS_12graph_view_tIlldLb0ELb0EvEEEESt5tupleIJN3rmm14device_uvectorINT_11vertex_typeEEES9_St8optionalINS6_INS7_11weight_typeEEEEEERKN4raft8handle_tERKS7_RNSF_6random8RngStateERKS9_mb+0x99d) [0x7f8040d704bd]
#4 in /datasets/vjawa/miniconda3/envs/cugraph_dgl_dev_oct

In [27]:
g.idtype

torch.int64

In [16]:
dir(g)

['__class__',
 '__contains__',
 '__copy__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_batch_num_edges',
 '_batch_num_nodes',
 '_canonical_etypes',
 '_dsttypes_invmap',
 '_edge_frames',
 '_etype2canonical',
 '_etypes',
 '_etypes_invmap',
 '_find_etypes',
 '_get_e_repr',
 '_get_n_repr',
 '_graph',
 '_idtype_str',
 '_init',
 '_is_unibipartite',
 '_node_frames',
 '_ntypes',
 '_pop_e_repr',
 '_pop_n_repr',
 '_reset_cached_info',
 '_set_e_repr',
 '_set_n_repr',
 '_srctypes_invmap',
 'add_edge',
 'add_edges',
 'add_nodes',
 'add_self_loop',
 'adj',
 'adj_sparse',
 'adjacency_matrix',
 'adjacency_matrix_scipy',
 'all_edges',
 'apply_edges',
 'apply_n